In [ ]:
pip install python-binance

In [ ]:
pip install websocket_client

In [ ]:
from binance.client import Client
from binance.websockets import BinanceSocketManager
import pandas as pd
import numpy as np
import websocket
import json
import pprint
from datetime import timedelta, datetime



In [ ]:
# Initialize our client and pass through the API key and secret
api_key = input("enter your public API key: ")
api_secret = input("enter your secret API key:")

client = Client(api_key, api_secret)


**Last 24 hour**

In [ ]:
def timestamp_N_days_ago(N=1):
    '''
    gets number of days and return the timestamp of n days ago
    parametr: int(number of days)
    return: timestamp
    '''   
    date_N_days_ago = datetime.now() - timedelta(days=N)
    date_N_days_ago_timestamp = date_N_days_ago.timestamp()
    return date_N_days_ago_timestamp


def convert_dataframe(data):
    '''
    convert 2d array with timstamp column in to datafram with date and time column
    parametr: 2d array
    return: datafram
    '''
    btc_df = pd.DataFrame(data, columns=['close_price', 'close_time'])
    btc_df['close_time'] = pd.to_datetime(btc_df['close_time'], unit='ms')
    return btc_df


bars = client.get_historical_klines('BTCUSDT', '1m', 
                                    str(timestamp_N_days_ago(1)), 
                                    limit=1000)

bars_arr = np.array(bars)

#select only close time and close price
bars_arr = bars_arr[:,[4,6]]
btc_24h_ago_df = convert_dataframe(bars_arr)

#write them on excel
btc_24h_ago_df.to_excel('BTCUSDT_24h_ago.xlsx')


In [ ]:
print(btc_24h_ago_df)


,close_price,close_time
0,46547.50000000,2021-02-12 14:59:59.999
1,46725.66000000,2021-02-12 15:00:59.999
2,46715.50000000,2021-02-12 15:01:59.999
3,46684.84000000,2021-02-12 15:02:59.999
4,46680.40000000,2021-02-12 15:03:59.999
...,...,...
1435,46884.29000000,2021-02-13 14:54:59.999
1436,46918.15000000,2021-02-13 14:55:59.999
1437,46972.76000000,2021-02-13 14:56:59.999
1438,46989.20000000,2021-02-13 14:57:59.999


**24 hours real time**

In [ ]:
#set the timestmp for 24 hour later for closing the stream
tmiestamp_close_ws = datetime.now() + timedelta(hours=24)

#the stream wss
SOCKET = "wss://stream.binance.com:9443/ws/btcusdt@kline_1m"

close_price = []
close_timestamp = []


def on_open(ws):
    print('opened connection')


def on_close(ws):
    print('closed connection')


def on_message(ws, message):
    #print('received message')
    json_message = json.loads(message)
    #pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_close = candle['x']
    close_p = candle['c']
    close_t = candle['T']

    if is_candle_close:
        close_price.append(float(close_p))
        close_timestamp.append(close_t)
        
        

        if tmiestamp_close_ws <= datetime.now() :
            ws.close()


ws = websocket.WebSocketApp(SOCKET, on_open=on_open,
                            on_close=on_close,
                            on_message=on_message)

ws.run_forever()


opened connection


In [ ]:
close_candle = []
#Convert 2 list in to 2d array
for i in range(len(close_price)): 
    close_candle.append([close_price[i], close_timestamp[i]])

btc_24h_real_time_df = convert_dataframe(close_candle)
print(btc_24h_real_time_df)


In [ ]:
#write them on excel
btc_24h_real_time_df.to_excel('BTCUSDT_24h_real_time.xlsx')